# Cleaning Folha Dataset

In [1]:
import pandas as pd
import numpy as np
import html
import re
from datasets import load_dataset
from datasets import Dataset

pd.set_option('display.max_rows', 120)

d:\Anaconda\envs\folha-scrapy\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Load Dataset and Overview

In [2]:
dataset_filepath = "../data/folha_2015_2023.zip"
folha = pd.read_csv(dataset_filepath)
folha.head()

,title,text,date,category,subcategory,link
0,"Lula diz que está 'lascado', mas que ainda tem...",Com a possibilidade de uma condenação impedir ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
1,"'Decidi ser escrava das mulheres que sofrem', ...","Para Oumou Sangaré, cantora e ativista malines...",2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
2,Três reportagens da Folha ganham Prêmio Petrob...,Três reportagens da Folha foram vencedoras do ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
3,Filme 'Star Wars: Os Últimos Jedi' ganha trail...,A Disney divulgou na noite desta segunda-feira...,2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
4,CBSS inicia acordos com fintechs e quer 30% do...,"O CBSS, banco da holding Elopar dos sócios Bra...",2017-09-10,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/10/1...


In [3]:
# number of rows and columns
print(f"Rows: {folha.shape[0]}")
print(f"Columns: {folha.shape[1]}")

Rows: 532727
Columns: 6


## 2. Checking columns

### 2.1. title

In [4]:
# NaN titles
print(f"NaN title rows: {folha['title'].isna().sum()}")
print(f"% NaN title rows: {folha['title'].isna().sum() / len(folha) * 100} %")

NaN title rows: 0
% NaN title rows: 0.0 %


In [5]:
# check for double spaces, tabs or line breaks
cond = folha['title'].str.contains(r"  +|\t+|\n+")
folha.loc[cond, 'title']

43        \n\t\tDa escola ao 1º emprego: como buscar um ...
160       \n\t\tComo diferenciar ruídos que indicam prob...
163       \n\t\tAdvogado explica as regras sobre venda d...
165       \n\t\tMoradores da zona oeste são os mais sati...
171       \n\t\tMoradores recuperam praça no Jardim das ...
                                ...                        
125343    Grécia inicia o pagamento de  6,25 bilhões ao ...
126322    Comissão da UE propõe empréstimo de curto praz...
145679    Moradores esperam por obras públicas  na 'Ilha...
501558    Morcheeba retorna ao Brasil para  Nublu Jazz F...
504758    ​    Para leitores, Carnaval de São Paulo sobr...
Name: title, Length: 3931, dtype: object

In [6]:
# check for double spaces, tabs or line breaks
folha.loc[cond, 'title'].tail(1).values[0]

'\u200b    Para leitores, Carnaval de São Paulo sobrepõe dinheiro ao bem-estar da população'

### 2.2. text

In [9]:
# NaN text
print(f"NaN text rows: {folha['text'].isna().sum()}")
print(f"% NaN text rows: {folha['text'].isna().sum() / len(folha) * 100} %")

NaN text rows: 32009
% NaN text rows: 6.008518434395103 %


In [10]:
# check for double spaces, tabs or line breaks
cond = folha['text'].apply(lambda x: str(x)).str.contains(r"  +|\t+|\n+")
folha.loc[cond, 'text'].tail(1).values[0]

'Confira o que acontece no capítulo desta terça\xa0(2) em "Tempos de Amar", novela de Alcides Nogueira:Em Morros Verdes, Maria Vitória (Vitória Strada) insiste para que Inácio (Bruno Cabrerizo) mantenha seus planos de ir para o Brasil. José Augusto (Tony Ramos) elogia Inácio. Inácio se despede de Maria Vitória.No Rio, Conselheiro (Werner Shünemann) termina o romance com Celeste Hermínia (Marisa Orth).\xa0Passa-se mais um mês. Inácio chega ao Rio de Janeiro. Em Morros Verdes, Maria Vitória torce por notícias de Inácio.No Rio, Vicente\xa0(Bruno Ferrari) tenta convencer Conselheiro a procurar Celeste Hermínia. Teodoro pensa em cortejar a fadista. Inácio começa a trabalhar com Geraldo (Jackson Antunes).                         Em Morros Verdes, Maria Vitória sente uma forte tontura e é observada pela governanta. Delfina (Letícia Sabatella) afirma a Tereza (Olivia\xa0Torres) que ela receberá uma grande quantia para se casar. No Rio, Reinaldo ( Cassio Gabus Mendes)\xa0insiste para que Lucind

In [11]:
text = folha.loc[cond, 'text'].tail(10).values[0]
test = re.sub(r"\s+", " ", text)
test

'A saga da personagem de Carol Duarte é repleta de grandes acontecimentos: Ivana se descobre trans, encara o processo de readequação de gênero, muda seu nome para Ivan e fica grávido.Desde a revelação da gravidez, na última sexta (29), espectadores passaram a fazer algumas contas para identificar quando ocorreu a fecundação.Desde que Ivan (enquanto ainda era Ivana) transou com Cláudio (Gabriel Stalffer) muitas coisas aconteceram na novela. Internautas calcularam que o tempo transcorrido na trama seja de aproximadamente um ano. "Ivana grávida depois de séculos que o namorado dela foi embora. Gente, que espermatozóide forte esse", escreveu uma internauta no Twitter. Gloria Perez explicou que a contagem do tempo é diferente. Em entrevista ao portal "Extra", a autora disse que "em novela uma noite ou um dia podem durar uma semana inteira até". Depois daquela noite, Cláudio disse que ficaria um ano fora do país e agora o personagem já deve voltar para o folhetim. Então, de fato, quanto temp

### 2.3. date

In [12]:
# NaN data
print(f"NaN date rows: {folha['date'].isna().sum()}")
print(f"% NaN date rows: {folha['date'].isna().sum() / len(folha) * 100} %")

NaN date rows: 30117
% NaN date rows: 5.653364668958022 %


### 2.4. category

In [14]:
# NaN categories
print(f"NaN category rows: {folha['category'].isna().sum()}")
print(f"% NaN category rows: {folha['category'].isna().sum() / len(folha) * 100} ")

NaN category rows: 0
% NaN category rows: 0.0 


In [15]:
len(folha['category'].value_counts().sort_values(ascending = False))

112

In [16]:
# number of rows per category
folha['category'].value_counts().sort_values(ascending = False)

category
colunas                                            88528
mercado                                            77447
poder                                              51463
mundo                                              42016
cotidiano                                          39639
ilustrada                                          38162
esporte                                            35389
celebridades                                       17627
opiniao                                            14228
equilibrioesaude                                   11783
televisao                                           9774
internacional                                       7775
paineldoleitor                                      6575
sao-paulo                                           6181
educacao                                            5088
colunistas                                          5047
blogs                                               4926
saopaulo              

In [17]:
# colunas x colunistas (seems to be the same thing)
print(folha.query("category == 'colunas'").iloc[2, -1])
print(folha.query("category == 'colunistas'").iloc[0, -1])

http://www1.folha.uol.com.br/colunas/marceloleite/2017/10/1925096-cientista-da-psicodelico-para-minicerebros-e-eles-gostam.shtml
https://f5.folha.uol.com.br/colunistas/cristina-padiglione/2023/05/galvao-bueno-nao-tenho-por-que-pedir-desculpas-ao-neymar.shtml


In [18]:
# restaurantes x comidas (seems to be the same thing)
print(folha.query("category == 'restaurantes'").iloc[0, -1])
print(folha.query("category == 'comida'").iloc[1, -1])

https://guia.folha.uol.com.br/restaurantes/2023/05/restaurante-quincho-serve-menu-degustacao-apenas-com-sobremesas-em-sp.shtml
http://www1.folha.uol.com.br/comida/2017/04/1872819-eleven-madison-park-de-ny-e-eleito-o-melhor-restaurante-do-mundo.shtml


In [19]:
# cinemas-e-series x cinema x 42a-mostra-internacional-de-cinema x 41a-mostra-internacional-de-cinema (seems to be the same thing)
print(folha.query("category == 'cinema-e-series'").iloc[2, -1])
print(folha.query("category == 'cinema'").iloc[4, -1])
print(folha.query("category == '42a-mostra-internacional-de-cinema'").iloc[0, -1])
print(folha.query("category == '41a-mostra-internacional-de-cinema'").iloc[1, -1])

https://f5.folha.uol.com.br/cinema-e-series/2023/05/barbie-ganha-novo-trailer-com-mais-detalhes-da-historia-da-boneca-entrando-no-mundo-real.shtml
https://guia.folha.uol.com.br/cinema/2023/05/velozes-e-furiosos-10-domina-as-salas-de-cinema-em-sp-que-tem-5-estreias.shtml
https://guia.folha.uol.com.br/42a-mostra-internacional-de-cinema/2018/10/central-do-brasil-tem-sessao-com-fernanda-montenegro-na-mostra-confira-outras-dicas-do-dia.shtml
https://guia.folha.uol.com.br/41a-mostra-internacional-de-cinema/2017/11/veja-tres-filmes-em-destaque-no-ultimo-dia-da-mostra-de-cinema.shtml


In [20]:
# sao-paulo x saopaulo x o-melhor-de-sao-paulo (seems to be the same thing)
print(folha.query("category == 'sao-paulo'").iloc[9, -1])
print(folha.query("category == 'saopaulo'").iloc[30, -1])
print(folha.query("category == 'o-melhor-de-sao-paulo'").iloc[12, -1])

https://agora.folha.uol.com.br/sao-paulo/2021/11/arvore-de-natal-na-ponte-estaiada-em-sp-sera-inaugurada-neste-sabado-27.shtml
http://www1.folha.uol.com.br/saopaulo/2017/10/1922945-padrao-geometrico-conhecido-como-chevron-ajuda-a-dar-movimento-aos-ambientes.shtml
http://www1.folha.uol.com.br/o-melhor-de-sao-paulo/2017/casa-e-decoracao/2017/08/1910839-jovens-dedicam-se-a-ceramica-artesanal-e-produzem-pecas-cheias-de-estilo.shtml


## 3. Clean Dataset

In [21]:
folha_filtered = folha.copy()

In [22]:
def group_categories(df):
    """
    Group similar categories into one.
    """
    df.loc[df['category'] == 'equilibrioesaude', 'category'] = 'equilibrio'
    df.loc[df['category'] == 'tv', 'category'] = 'televisao'
    df.loc[df['category'] == 'mundo', 'category'] = 'internacional'
    df.loc[df['category'] == 'cinema', 'category'] = 'cinema-e-series'
    df.loc[df['category'] == 'equilibrio', 'category'] = 'equilibrio-e-saude'

    return df

In [23]:
def clean_text(df, column):
    """
    Cleans text column, removing tabs, line breaks, double spaces, unicode and html strings. Removes NaN values.
    """

    # remove NaN values
    df.dropna(subset = column, inplace = True)

    # remove tabs and line breaks
    df[column] = df[column].str.replace(r"\t+|\n+", "")

    # remove double spaces and strip
    df[column] = df[column].str.strip() \
                        .apply(lambda text: re.sub(r"\s+", " ", text))

    # remove html strings
    df[column] = df[column].apply(lambda text: html.unescape(text))

    return df

### 3.1. Filter categories

In [24]:
categories_to_drop = [
    "folhinha",
    "webstories",
    "seminariosfolha",
    "banco-de-dados",
    "sobretudo",
    "voceviu",
    "ilustrissima",
    "blogs",
    "colunistas",
    "sao-paulo",
    "saopaulo",
    "paineldoleitor",
    "opiniao",
    "ilustrada",
    "colunas"
]

In [25]:
aux = pd.DataFrame(folha['category'].value_counts().rename('qtd').sort_values(ascending=False))
aux = aux.query("qtd > 1000").reset_index()
aux = aux.query("~category.isin(@categories_to_drop)")
filtered_categories = aux['category'].unique()
filtered_categories

array(['mercado', 'poder', 'mundo', 'cotidiano', 'esporte',
       'celebridades', 'equilibrioesaude', 'televisao', 'internacional',
       'educacao', 'grana', 'tec', 'turismo', 'ambiente', 'ciencia',
       'cinema-e-series', 'musica', 'tv', 'empreendedorsocial',
       'podcasts', 'comida', 'cinema', 'equilibrio'], dtype=object)

In [26]:
print(f"Before filtering categories: {len(folha_filtered)}")
folha_filtered = folha_filtered.query("category.isin(@filtered_categories)").copy()
print(f"After filtering categories: {len(folha_filtered)}")

Before filtering categories: 532727
After filtering categories: 332930


### 3.2. Clean text

In [27]:
def clean_dataset(df):
    """
    Pipeline to clean dataset.
    """
    print(f"Rows in dataset: {len(df)}")
    # cleans title
    df = clean_text(df, 'title')
    print(f"Rows in dataset after title cleaning: {len(df)}")

    # cleans text
    df = clean_text(df, 'text')
    print(f"Rows in dataset after text cleaning: {len(df)}")

    # group categories
    df = group_categories(df)

    return df

In [28]:
# clean and export
folha_filtered = clean_dataset(folha_filtered)

Rows in dataset: 332930
Rows in dataset after title cleaning: 332930
Rows in dataset after text cleaning: 308862


### 3.3. Filter Columns

In [29]:
folha_filtered = folha_filtered.loc[:, ["title", "text", "date", "category", "link"]]
folha_filtered.head()

,title,text,date,category,link
0,"Lula diz que está 'lascado', mas que ainda tem...",Com a possibilidade de uma condenação impedir ...,2017-09-10,poder,http://www1.folha.uol.com.br/poder/2017/10/192...
2,Três reportagens da Folha ganham Prêmio Petrob...,Três reportagens da Folha foram vencedoras do ...,2017-09-10,poder,http://www1.folha.uol.com.br/poder/2017/10/192...
4,CBSS inicia acordos com fintechs e quer 30% do...,"O CBSS, banco da holding Elopar dos sócios Bra...",2017-09-10,mercado,http://www1.folha.uol.com.br/mercado/2017/10/1...
5,"Em encontro, Bono pergunta a Macri sobre argen...","O vocalista da banda irlandesa U2, Bono, fez u...",2017-09-10,internacional,http://www1.folha.uol.com.br/mundo/2017/10/192...
6,"Posso sair do Brasil quando e como quiser, diz...",O italiano Cesare Battisti disse nesta segunda...,2017-09-10,poder,http://www1.folha.uol.com.br/poder/2017/10/192...


### 3.4. Check changes

In [30]:
qty = folha_filtered['category'].value_counts()
pct = folha_filtered['category'].value_counts(normalize = True)
qty_pct = pd.DataFrame([qty, pct]).T
qty_pct['count'] = qty_pct['count'].astype(int)
qty_pct

,count,proportion
category,,
mercado,59022,0.191095
poder,49976,0.161807
internacional,48657,0.157536
cotidiano,38608,0.125001
esporte,34705,0.112364
celebridades,17368,0.056232
equilibrio-e-saude,12681,0.041057
televisao,12133,0.039283
educacao,4962,0.016065


## 4. Export and push to hub

### 4.1. Export to CSV (.zip)

In [ ]:
# Export file
# filename = 'folha_2015_2023_clean_cats'
# print(f"Exporting file {filename}")
# compression_options = dict(method='zip', archive_name=f'{filename}.csv')
# folha_filtered.to_csv(f'../data/{filename}.zip', compression=compression_options, index = False)

### 4.2. Export to Hub

In [ ]:
# dataset_folha_final = Dataset.from_pandas(folha_filtered[["title", "text", "date", "category", "link"]])
# dataset_folha_final = dataset_folha_final.remove_columns("__index_level_0__")

In [ ]:
# print(dataset_folha_final)

Dataset({
    features: ['title', 'text', 'date', 'category', 'link'],
    num_rows: 308862
})


In [ ]:
# dataset_folha_final.push_to_hub("iara-project/news-articles-ptbr-dataset")

Pushing dataset shards to the dataset hub: 100%|██████████| 3/3 [02:35<00:00, 51.90s/it]
